## IBM Applied Data Science Capstone Assignment Week 5 
### Week 5 Coding Report - Finding the Cluster(s) For Opening Vietnamese Restaurant in Kuala Lumpur, Malaysia
* HTML Scrapping of Wikipedia dara to build a dataframe of Kuala Lumpur suburbs and neighborhood with it latitude and longtitude co-ordinates
* Using Four Square API to build a venue information on Vietnamese Restaurant in Kuala Lumpur neighborhoods
* Data Analysis of Four Square API output.
* Data Wrangling to clean and format into relevant data. 
* Use Visual Map to decide on the area in Kuala Lumpur City that optimum for opening a new Vietnamese Restaurant.

#### Initialize Python Library 

In [202]:
# library to handle data in a vectorized manner
import numpy as np 

 # library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# convert an address into latitude and longitude values for Four Square API
# !conda install -c conda-forge geopy --yes
import geocoder
from geopy.geocoders import Nominatim 


# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# BeautifulSoup - htlm parser
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print("Library imported.")

Library imported.


#### Retrieving Kuala Lumpur, Malaysia Geo Position

In [203]:
# Retrieving the coordinate of Kuala Lumpur, Malaysia
address = "Kuala Lumpur, Malaysia"

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Kuala Lumpur: {}, {} " .format(latitude, longitude))

The latitude and longitude of Kuala Lumpur: 3.1516636, 101.6943028 


### Four Square API to collect information on Vietnam Restaurant within 20km of KL City Geo-position

In [204]:
CLIENT_ID = 'TILBQRF2U0VD1XLHXL3F1W05UGZV53GUP30VNU0HFSBMYJKL' # your Foursquare ID
CLIENT_SECRET = 'HD2XVCOKAQNYAZL5LO4E0IGX3IN4GKEV1SLOKO5GRNLM0OWF' # your Foursquare Secret
VERSION = '20180604'
# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [222]:
# Using Four Square API call to get the Vietnamese Restaurant Information within 20km radius from KL City Geo-position

search_query = 'Vietnamese'
radius = 10000
LIMIT = 150
viet_restaurant = []

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# Update the vietnamese restauranr data frame with relevant columns
list = ["name", "categories", "lat", "lng","city", "state", "formattedAddress", "postalCode", "neigborhood"]
viet_restaurant = pd.DataFrame(dataframe_filtered, columns=list)
viet_restaurant


,name,categories,lat,lng,city,state,formattedAddress,postalCode,neigborhood
0,Vietnamese Hawker Stall,Vietnamese Restaurant,3.153693,101.707811,Kuala Lumpur,Federal Territory of Kuala Lum,"[Kuala Lumpur, Federal Territory of Kuala Lum,...",NaN,NaN
1,Vietnamese Delicacy,Vietnamese Restaurant,3.143604,101.710960,NaN,NaN,"[(Inside Big Food Street@Sungei Wang), Malaysia]",NaN,NaN
2,Vinh City Vietnamese Cuisine,Vietnamese Restaurant,3.144222,101.710826,Kuala Lumpur,Kuala Lumpur,"[Level 3, Sungei Wang, Kuala Lumpur, Kuala Lum...",NaN,NaN
3,Du Viet Vietnamese Cuisine,Vietnamese Restaurant,3.157303,101.711464,Kuala Lumpur,Kuala Lumpur,"[Suria KLCC (LC401, 4th Level), 50088 Kuala Lu...",50088,NaN
4,Hot Pot Vietnamese Food,Vietnamese Restaurant,3.146045,101.710819,Kuala Lumpur,Kuala Lumpur,"[Kuala Lumpur, Kuala Lumpur, Malaysia]",NaN,NaN
5,the besr Vietnamese noodle in town,Vietnamese Restaurant,3.153330,101.707664,NaN,NaN,[Malaysia],NaN,NaN
6,Daviet vietnamese restaurant,None,3.157248,101.711330,NaN,NaN,"[Surya KLCC, Malaysia]",NaN,NaN
7,Vietnamese Pork Chop Rice,Vietnamese Restaurant,3.117273,101.604120,Petaling Jaya,Selangor,"[Mayang Oasis, 47301 Petaling Jaya, Selangor, ...",47301,NaN
8,Ori vietnamese food,Asian Restaurant,3.156855,101.709545,NaN,NaN,[Malaysia],NaN,NaN
9,Ara Vietnamese Noodles,Vietnamese Restaurant,3.130415,101.621217,Petaling Jaya,Selangor,"[45 Jalan SS22/11 (Damansara Jaya), 47400 Peta...",47400,NaN


### Data Observation and Analysis

In [223]:
viet_restaurant.shape

(33, 9)

The filtered data frame (from Four Square API JSON output) return 33 row of data with nine category of information

In [224]:
# Analysis of the quality of data gathered from Four Square API call - group by state
viet_restaurant_group = viet_restaurant.groupby(["state"], as_index=False)
viet_restaurant_group.count()

,state,name,categories,lat,lng,city,formattedAddress,postalCode,neigborhood
0,Federal Territory of Kuala Lum,2,2,2,2,2,2,1,0
1,KL,2,2,2,2,2,2,2,0
2,Kuala Lumpur,11,11,11,11,11,11,8,0
3,Selangor,4,3,4,4,4,4,2,0


It was found from the above analysis that data gathered from Four Square is incomplete and the label in neigborhood column is totally empty which rendered the information in the neigborhood columns useless. Similary, majority of the data in postalCode is incomplete. There is 4 restaurant based on state columns is located in Selangor which is outside of Kuala Lumpur city area, the 4 restaurant can be dropped later in the data wrangling section below. 

In [225]:
# Analysis of the quality of data gathered from Four Square API call - group by categories
viet_restaurant_group = viet_restaurant.groupby(["categories"], as_index=False)
viet_restaurant_group.count()

,categories,name,lat,lng,city,state,formattedAddress,postalCode,neigborhood
0,Asian Restaurant,2,2,2,0,0,2,0,0
1,Coffee Shop,2,2,2,2,2,2,2,0
2,Strip Club,1,1,1,0,0,1,0,0
3,Supermarket,1,1,1,1,1,1,1,0
4,Vietnamese Restaurant,25,25,25,16,15,25,11,0


Out of 33 row of data, 25 labelled as Vietnamese Restaurant data, 6 in other categories and 2 is has missing labelled.
Categories such as Strip Club, Supermarket is not relevant and will be drop in Data Wrangling Section below. The Asian Restaurant and Coffee Shop categories will be investigate and replace with Vietnamese Restaurant label if it is found to be relevant in Data Wrangling Section below.

In [226]:
# Analysis of the quality of data gathered from Four Square API call - group by city
viet_restaurant_group = viet_restaurant.groupby(["city"], as_index=False)
viet_restaurant_group.count()

,city,name,categories,lat,lng,state,formattedAddress,postalCode,neigborhood
0,Genting Klang,1,1,1,1,0,1,1,0
1,Kepong,2,2,2,2,2,2,2,0
2,Kuala Lumpur,13,13,13,13,13,13,9,0
3,Petaling Jaya,4,3,4,4,4,4,2,0


Petaling Jaya is not part of Kuala Lumpur which can be dropped in Data Wrangling Section below. Genting Klang and Kepong is part of Kuala Lumpur City and the label will be replaced in Data Wrangling Section with Kuala Lumpur for consistency.

### Data Wrangling Section

In [227]:
# Data Wrangling - fill up missing data and dropping unnecessary columns in viet_restaurant_group data frame

# Global change all blank, none data in data frame to NaN 
viet_restaurant.fillna(value=pd.np.nan, inplace=True)

# Replace NaN data in city column with Kuala Lumpur and other with Kuala Lumpur
viet_restaurant.city.replace(np.nan,"Kuala Lumpur",inplace=True, regex = True)
viet_restaurant.city.replace("Genting Klang","Kuala Lumpur",inplace=True, regex = True)
viet_restaurant.city.replace("Kepong","Kuala Lumpur",inplace=True, regex = True)

# Replace NaN data in categories column with Vietnamese Restaurant
viet_restaurant.categories.replace(np.nan,"Vietnamese Restaurant",inplace=True, regex = True)

# Changing categories Asian Restaurant and Coffee Shop to Vietnamese Restaurant
viet_restaurant.categories.replace("Asian Restaurant", "Vietnamese Restaurant",inplace=True, regex=True)
viet_restaurant.categories.replace("Coffee Shop", "Vietnamese Restaurant",inplace=True, regex=True)

# Dropping row where the state is Selangor which is not part of Kuala Lumpur City
viet_restaurant = viet_restaurant[viet_restaurant.state != "Selangor"] 

# Dropping row with categories not equal Vietnamese Restaurant
viet_restaurant = viet_restaurant[viet_restaurant.categories == "Vietnamese Restaurant"]

# Dropping columns formattedAddress, neigborhood, and postalCode
new_list = ["name", "categories", "lat", "lng","city"]
viet_restaurant = pd.DataFrame(viet_restaurant, columns=new_list)

# Rename columns in data frame
viet_restaurant.rename(columns={'name':'Restaurant Name','categories':'Categories','lat': 'Latitude','lng':'Longitude',
                                'city':'City'}, inplace = True)
viet_restaurant

,Restaurant Name,Categories,Latitude,Longitude,City
0,Vietnamese Hawker Stall,Vietnamese Restaurant,3.153693,101.707811,Kuala Lumpur
1,Vietnamese Delicacy,Vietnamese Restaurant,3.143604,101.710960,Kuala Lumpur
2,Vinh City Vietnamese Cuisine,Vietnamese Restaurant,3.144222,101.710826,Kuala Lumpur
3,Du Viet Vietnamese Cuisine,Vietnamese Restaurant,3.157303,101.711464,Kuala Lumpur
4,Hot Pot Vietnamese Food,Vietnamese Restaurant,3.146045,101.710819,Kuala Lumpur
5,the besr Vietnamese noodle in town,Vietnamese Restaurant,3.153330,101.707664,Kuala Lumpur
6,Daviet vietnamese restaurant,Vietnamese Restaurant,3.157248,101.711330,Kuala Lumpur
8,Ori vietnamese food,Vietnamese Restaurant,3.156855,101.709545,Kuala Lumpur
10,Y Thu Wendy Authentic Vietnamese,Vietnamese Restaurant,3.159951,101.713766,Kuala Lumpur
11,Kah Hing Vietnamese Restaurant,Vietnamese Restaurant,3.189387,101.652780,Kuala Lumpur


In [228]:
viet_restaurant.shape

(27, 5)

The original Data Frame consist of 33 row of data and the cleaned data frame consist of 27 row of data. 6 row data were dropped - 4 which is from the state of Selangor (which is outside of KL City), 1 Strip Club and 1 Supermarket which is not relevant to Vietnamese Restaurant.

In [229]:
viet_restaurant.to_csv("viet_restaurant.csv", index=False)

In [230]:
viet_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='KL City Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(viet_restaurant_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(viet_restaurant_map)

# display map
display(viet_restaurant_map)

### From the visual map display, it is obvious from visual observation that the opportunity for Vietnamese Restaurant lies within Kuala Lumpur City center on the western, south and northern. Majority of the Vietnamese Restaurant is located on the eastern part of Kuala Lumpur city center. Since Python notebook cannot display Map, the Map will be ported into the report and presentation. The visual of the Map and explanation will be done in the presentation and report.

In [231]:
# Storing KL_Township map as HTML file
viet_restaurant_map.save("viet_restaurant_map.html")